In [ ]:
import pandas as pd
import numpy as np
!pip install -U sentence-transformers
from IPython.display import clear_output
from sentence_transformers import SentenceTransformer
import pickle
clear_output()

In [ ]:
###### Models ######
models_names = ["all-MiniLM-L6-v2","all-MiniLM-L12-v2"]
###### Datasets ######
datasets = ['TC02_20news', 'TC14_wos']
for dataset in datasets:
  df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
  df = df.dropna()
  for model_name in models_names:
    model = SentenceTransformer(f"sentence-transformers/{model_name}")
    textos = df['text'].values.tolist()
    embeddings = model.encode(textos)
    with open(f'Arquivos TCC/embeddings/sbert/{model_name}_{dataset}.pkl', "wb") as fOut:
      pickle.dump({'sentences': textos, 'embeddings': embeddings, 'labels': df['label']}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

clear_output()

In [ ]:
#umap para redução da dimensão do embedding
!pip install umap-learn
from umap import UMAP
import pandas as pd
import numpy as np
import pickle
from IPython.display import clear_output
clear_output()
datasets = ['TC02_20news', 'TC14_wos']
models_names = ["all-MiniLM-L6-v2","all-MiniLM-L12-v2"]
for model_name in models_names:
  for dataset in datasets:
    embeddings_path = f'Arquivos TCC/embeddings/{model_name}_{dataset}.pkl'
    with open(embeddings_path, "rb") as fIn:
      stored_data = pickle.load(fIn)
      stored_sentences = stored_data['sentences']
      stored_embeddings = stored_data['embeddings']
      stored_labels = stored_data['labels']
      umap_model = UMAP(
              n_components=30,
              min_dist=0.01,
              random_state=5,
              n_neighbors=20,
              metric='cosine',
              low_memory=True
          )
      embeddings_umap = umap_model.fit_transform(stored_embeddings)
      with open(f'Arquivos TCC/embeddings/umap/{model_name}_{dataset}.pkl', "wb") as fOut:
        pickle.dump({'sentences': stored_sentences, 'umap_embeddings': embeddings_umap, 'labels': stored_labels}, fOut, protocol=pickle.HIGHEST_PROTOCOL)